# **xgboost_timeseries_modeling_template**

---

<br><br><br><br>


## **Variable Inputs**

---


In [1]:
input_file_path = ''


## **Todo List**

-   TODO: more structure around the naming of dataframes in different steps of the process:2023-03-07

-   TODO: abstract certain functions into fp-data-toolbox :2023-03-07

<br>


# **Environment Setup**


In [2]:
# imports
import os
import dotenv
from fp_data_toolbox import eda, notifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
# custom imports
# visualization.set_plotly_defaults()
notifier.setup()  # Enable for windows toast notifications on Jupyter cell complete
# env variables
dotenv.load_dotenv()
df = pd.DataFrame()  # creating empty dataframe variable
params = {}  # creating empty parameters dictionary
# Magics env settings...
%matplotlib inline


<br><br>


## **Objectives**

---


example of XGBoost ML timeseries prediction pipeline


<br><br>


### **Visualizing ETL pipeline (example)**


In [3]:
import seaborn as sns

In [4]:
plt.style.use('fivethirtyeight')
color_pal = sns.color_palette()
color_pal


[(0.0, 0.5607843137254902, 0.8352941176470589),
 (0.9882352941176471, 0.30980392156862746, 0.18823529411764706),
 (0.8980392156862745, 0.6823529411764706, 0.2196078431372549),
 (0.42745098039215684, 0.5647058823529412, 0.30980392156862746),
 (0.5450980392156862, 0.5450980392156862, 0.5450980392156862),
 (0.5058823529411764, 0.058823529411764705, 0.48627450980392156)]

## **Function Definition**


### **Data Cleaning**


In [5]:
def replace_with_null(df):
    for col in df.columns:
        if df[col].dtype.name == 'category':
            df[col] = df[col].replace([-1, 0, 'UNK', 'NULL', 'NaN'], np.nan)
        if df[col].dtype.name == 'object':
            df[col] = df[col].replace([-1, 0, 'UNK', 'NULL', 'NaN'], np.nan)
        if df[col].dtype.name == 'float64':
            df[col] = df[col].replace([-1, 0, 'UNK', 'NULL', 'NaN'], np.nan)
    return df


## **Query Data**


In [6]:
from fredapi import Fred
# pass in fred api key
fred_key = os.environ.get('FREDAPI_KEY')
# fred_key = 'ed0a736bb61e29e91648917fec70e2e6'


### 1. Create the FRED object


In [7]:
fred = Fred(api_key=fred_key)

### 2. Search Fred

-   for economic data


In [8]:
# fred_search_prompt = 'unemployment rate state'
# fred_search_results = fred.search(
#     fred_search_prompt,
#     filter=('frequency', 'Monthly'),
#     order_by='popularity'
#     )
# fred_search_results = fred_search_results.query(
#     'seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"'
#     )
# fred_search_results = fred_search_results.loc[fred_search_results['title'].str.contains('Unemployment Rate')]

# fred_search_results.head()


<br><br><br>


### **Pull raw data**


In [9]:
# df_sp = fred.get_series('SP500')
# df_sp.plot(figsize=(10,5), title = 'S&P 500', lw=2)


In [10]:
# unemployment_results = fred.search('unemployment', order_by='popularity')
# unemployment_results.head()


In [11]:
# df_unrate = fred.get_series('UNRATE')
# df_unrate.plot(figsize=(10,5), title = 'Unemployment Rate', lw=2)


### 4. Pull and join multiple data series


In [12]:
fred_search_prompt = 'unemployment rate state'
fred_search_results = fred.search(
    fred_search_prompt,
    filter=('frequency', 'Monthly'),
    order_by='popularity'
)
fred_search_results = fred_search_results.query(
    'seasonal_adjustment == "Seasonally Adjusted" \
        and units == "Percent" \
        and title.str.contains("Unemployment Rate") \
        and not index.str.contains("LRUN") \
        and not index.str.contains("LRH") \
        and not index.str.contains("LNS") \
        and not index.str.contains("LAS") \
        and not index.str.contains("U2RATE") \
        and not index.str.contains("M089") \
        and not index.str.contains("PRUR") \
        and not index.str.contains("CMWRUR") \
        and not index.str.contains("CWSTUR") \
        and not index.str.contains("CSOUUR") \
        and not index.str.contains("CNEWUR") \
    '
)


In [13]:
# how many states are we pulling data for?
len(fred_search_results)


53

In [14]:
def daily_fredapi_buffer(df_input_search_results, file_name):
    """
    Saves a daily version of a Pandas DataFrame as a Parquet file in the same directory, 
    only if a file of the same name with a daily timestamp doesn't already exist.

    Used to prevent excessive API calls when testing data pipelines. 

    Parameters:
        df (Pandas DataFrame): The DataFrame to be saved.
        file_name (str): The base name of the file.

    Returns:
        None
    """
    import os
    import datetime as dt
    import fredapi

    # Check if a file with the same name and daily timestamp already exists
    timestamp = dt.datetime.now().strftime('%Y%m%d')
    file_path = f"{file_name}-{timestamp}.parquet"

    # ---------------------------------
    # break logic here
    if os.path.exists(file_path):
        print(f"File {file_path} already exists. Skipping save operation.")
        df = pd.read_parquet(file_path)
        return df

    # ---------------------------------

    # pull from api
    all_results = []
    for myid in df_input_search_results.index:
        results = fred.get_series(myid)
        results = results.to_frame(name=myid)
        all_results.append(results)

    df = pd.concat(all_results, axis=1)

    # ---------------------------------
    # Save the DataFrame as a Parquet file
    df.to_parquet(file_path)
    print(f"DataFrame saved as {file_path}.")
    return df


df_unemp_states = daily_fredapi_buffer(
    df_input_search_results=fred_search_results,
    file_name='df_unemp_states',
)


File df_unemp_states-20230418.parquet already exists. Skipping save operation.


In [15]:
# ---------------------------------
df = df_unemp_states
# ---------------------------------


### **Data Cleaning**


In [16]:
# filtering on relevant data where state level unemployment rate made available
state_data_start_dt = '1976-01-01'  # known from previous eda
df = df.query(f'index >= "{state_data_start_dt}"')
print('dataset filtered to ts >=', df.index.min())


dataset filtered to ts >= 1976-01-01 00:00:00


In [17]:
for col in df.columns:
    df[col].replace('NaN', None)

df = replace_with_null(df)


C:\Users\Fred\AppData\Local\Temp\ipykernel_8460\1870194956.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].replace([-1, 0, 'UNK', 'NULL', 'NaN'], np.nan)


In [18]:
ur_cols = [col for col in df.columns if col[-2:] == 'UR']
for col in ur_cols:
    df[f'{col}_1_LAG'] = df[col].shift(-1, freq='MS')
    df[col] = df[f'{col}_1_LAG']
    df.drop(columns=f'{col}_1_LAG', inplace=True)


C:\Users\Fred\AppData\Local\Temp\ipykernel_8460\1884664105.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{col}_1_LAG'] = df[col].shift(-1, freq='MS')
C:\Users\Fred\AppData\Local\Temp\ipykernel_8460\1884664105.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[f'{col}_1_LAG']
C:\Users\Fred\AppData\Local\Temp\ipykernel_8460\1884664105.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

In [19]:
df.tail()


,UNRATE,CAUR,NYUR,TXUR,FLUR,OHUR,PAUR,ALUR,MIUR,MAUR,...,HIUR,RIUR,NJUR,DEUR,IDUR,SDUR,NEUR,NHUR,VTUR,M08311USM156SNBR
2022-11-01,3.6,4.1,4.1,3.8,2.7,4.1,4.3,2.6,4.3,3.7,...,3.7,3.2,3.3,4.6,2.8,2.2,2.7,2.9,3.0,NaN
2022-12-01,3.5,4.2,4.2,3.9,2.6,4.0,4.3,2.6,4.3,3.5,...,3.6,3.1,3.4,4.6,2.7,2.1,2.5,2.8,2.9,NaN
2023-01-01,3.4,4.3,4.2,4.0,2.6,3.9,4.4,2.5,4.3,3.7,...,3.6,3.1,3.5,4.6,2.6,2.1,2.3,2.7,2.8,NaN
2023-02-01,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-01,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br><br><br>


## **Define Target for Model**


In [20]:
TARGET = 'UNRATE'  # the national unemployment rate
tgt_cols = [col for col in df.columns if col[:] == TARGET]


<br><br><br>


## **Feature Creation**


In [21]:
# feature creation here

def extract_datetime_features(df, date_col):
    """
    Extracts various datetime features from a pandas dataframe column and returns the modified dataframe.

    Parameters
    ----------
    df : pandas.DataFrame
        The input dataframe containing the date column to extract features from.
    date_col : str
        The name of the date column to extract features from. If 'index', the function will use the index as the date column.

    Returns
    -------
    pandas.DataFrame
        The modified dataframe with the extracted datetime features added as columns.

    Notes
    -----
    This function extracts the following datetime features: day of week, day of month, day of quarter, day of year,
    week of month, week of quarter, week of year, month of year, quarter of year, and year. All extracted features are
    added as new columns to the input dataframe.

    """
    if date_col == 'index':
        df['date_index'] = df.index
        date_col = 'date_index'

    # Extract year, month, week of year, and week of quarter
    df['DAY_OF_WEEK_CAL'] = df[date_col].dt.dayofweek
    df['DAY_OF_MNTH_CAL'] = df[date_col].dt.day
    df['DAY_OF_QTR_CAL'] = df[date_col].dt.dayofyear - \
        ((df[date_col].dt.quarter - 1) * 90)
    df['DAY_OF_YEAR_CAL'] = df[date_col].dt.dayofyear
    df['WEEK_OF_MNTH_CAL'] = (df[date_col].dt.day - 1) // 7 + 1
    df['WEEK_OF_QTR_CAL'] = df[date_col].dt.week % 13
    df['WEEK_OF_YEAR_CAL'] = df[date_col].dt.week
    df['MNTH_OF_YEAR_CAL'] = df[date_col].dt.month
    df['QTR_OF_YEAR_CAL'] = df[date_col].dt.quarter
    df['YEAR_CAL'] = df[date_col].dt.year

    # Convert all new features to int64 data type
    df = df.astype({
        'DAY_OF_WEEK_CAL': 'int64',
        'DAY_OF_MNTH_CAL': 'int64',
        'DAY_OF_QTR_CAL': 'int64',
        'DAY_OF_YEAR_CAL': 'int64',
        'WEEK_OF_MNTH_CAL': 'int64',
        'WEEK_OF_QTR_CAL': 'int64',
        'WEEK_OF_YEAR_CAL': 'int64',
        'MNTH_OF_YEAR_CAL': 'int64',
        'QTR_OF_YEAR_CAL': 'int64',
        'YEAR_CAL': 'int64',
    })

    # Return dataframe with new datetime features merged on the datetime index
    return df


df = extract_datetime_features(
    df,
    'index'
)


C:\Users\Fred\AppData\Local\Temp\ipykernel_8460\179651.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_index'] = df.index
C:\Users\Fred\AppData\Local\Temp\ipykernel_8460\179651.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DAY_OF_WEEK_CAL'] = df[date_col].dt.dayofweek
C:\Users\Fred\AppData\Local\Temp\ipykernel_8460\179651.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [22]:
def create_lag_and_window_features(df, target_col, lag_list=[1, 2, 3], window_list=[30, 90, 180]):
    """
    Creates lag and rolling window features for a given target column in a pandas DataFrame.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the target column.
    target_col : str
        The name of the target column.
    lag_list : list, optional (default=[1,2,3])
        A list of integers representing the lags to create.
    window_list : list, optional (default=[30,90,180])
        A list of integers representing the windows to create rolling features for.

    Returns:
    --------
    pandas.DataFrame
        The DataFrame with the added lag and rolling window features.
    """
    # Create lag features
    for lag in lag_list:
        df[f'{target_col}_{lag}_LAG'] = df[target_col].shift(lag, freq='M')

    # Create rolling window features
    for window in window_list:
        df[f'{target_col}_{window}_WIN_MEAN_LAG'] = df[target_col].rolling(
            window=f'{window}D').mean()
        df[f'{target_col}_{window}_WIN_MIN_LAG'] = df[target_col].rolling(
            window=f'{window}D').min()
        df[f'{target_col}_{window}_WIN_MAX_LAG'] = df[target_col].rolling(
            window=f'{window}D').max()
        df[f'{target_col}_{window}_WIN_STD_LAG'] = df[target_col].rolling(
            window=f'{window}D').std()

    return df


df = create_lag_and_window_features(
    df,
    TARGET,
)


<br><br><br>


## **Define training features for model**


In [23]:
# get unemployment rate column names for each individual state
ur_cols = [col for col in df.columns if col[-2:] == 'UR']
cal_cols = [col for col in df.columns if col[-3:] == 'CAL']
lag_cols = [col for col in df.columns if col[-3:] == 'LAG']


In [24]:
# Define features for training here
FEATURES = \
    ur_cols + cal_cols + lag_cols + [

        # Time series features
        # ---------------------------------


        # ---------------------------------
        # Categorical features


        # ---------------------------------
        # Boolean features

        # ---------------------------------
        # ---------------------------------
        # ---------------------------------
    ]

# Define target features here


In [25]:
# specify the desired column order
column_order = \
    ur_cols + \
    cal_cols + \
    lag_cols + \
    tgt_cols

# reindex the dataframe with the new column order
df = df.reindex(columns=column_order)

# df.info()


## **Data Profiling (Pre-modeling)**


In [26]:
# plot states unemployment rates
df_viz = df[ur_cols]
px.line(df_viz)

In [27]:
# df.head(5)

<br><br><br>


#### **Visualize Feature / Target Relationship**


In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
ax.set_title = ('TARGET by month in half')
sns.boxplot(
    data=df,
    x='MNTH_OF_YEAR_CAL',
    y=TARGET,
)


In [ ]:
# stop


## **Forecast horizon + Train / Test split**


#### **Define forecast time horizon variables**


In [ ]:
import datetime as dt

months_fwd_fc_horizon = 48

today = dt.date.today()
first_of_next_month = today.replace(day=1, month=today.month+1)
if first_of_next_month.month == 1:
    first_of_next_month = first_of_next_month.replace(
        year=first_of_next_month.year+1)

fc_start_date_str = first_of_next_month
# fc_start_date_str = ''
fc_start_date = pd.Timestamp(fc_start_date_str)
fc_end_date = fc_start_date + \
    pd.DateOffset(months=months_fwd_fc_horizon)  # USER INPUT
fc_date_range = pd.date_range(
    start=fc_start_date, end=fc_end_date, freq='MS')  # month start frequency
# reference this link for frequency charts:
# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases


## **Train / Test Split**


In [ ]:
test_train_split_date = fc_start_date - \
    pd.DateOffset(months=256)  # USER INPUT variable of split date

df_train = df.loc[df.index < test_train_split_date]
df_test = df.loc[df.index >= test_train_split_date]


In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
df_train[TARGET].plot(
    ax=ax, style='.', label='Training Set', title='Train/Test Split')
df_test[TARGET].plot(ax=ax, style='.', label='Test Set')
ax.axvline(test_train_split_date, color='black', ls='--')
ax.legend(['Training Set', 'Test Set'])
plt.show()


## **Experiments with creating test models**


In [ ]:
# convert columns to proper datatypes for ML learning

# df_train = convert_columns_for_ml(df_train)
# df_test = convert_columns_for_ml(df_test)


In [ ]:
X_train = df_train[FEATURES]
y_train = df_train[TARGET]

X_test = df_test[FEATURES]
y_test = df_test[TARGET]


### **Hyperparameter Optimization WIP**


In [ ]:
# # preprocessing for hyperparameter tuning
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y_train = le.fit_transform(y_train)

# %%
# # import packages for hyperparameters tuning
# from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
# from sklearn.metrics import accuracy_score

# #initialize the domain space for hyperparameters to be optimized
# space={
#         'n_estimators': 100,
#         'max_depth': hp.quniform('max_depth', 3, 12, 1),
#         # 'gamma': hp.uniform ('gamma', 0.2,5),
#         # 'reg_alpha' : hp.quniform('reg_alpha', 0.001, 0.1, 1),
#         # 'reg_lambda' : hp.uniform('reg_lambda', 0,1),
#         # 'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
#         # 'min_child_weight' : hp.quniform('min_child_weight', 1, 8, 1),
#         'seed': 0,
#     }

# def objective(space):
#     clf=xgb.XGBClassifier(
#                     n_estimators =space['n_estimators'],
#                     max_depth = int(space['max_depth']),
#                     # gamma = space['gamma'],
#                     # reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
#                     # colsample_bytree=int(space['colsample_bytree'])
#                     )

#     evaluation = [( X_train, y_train), ( X_test, y_test)]

#     clf.fit(X_train, y_train,
#             eval_set=evaluation, eval_metric="auc",
#             early_stopping_rounds=10,verbose=False)


#     pred = clf.predict(X_test)
#     accuracy = accuracy_score(y_test, pred>0.5)
#     print ("SCORE:", accuracy)
#     return {'loss': -accuracy, 'status': STATUS_OK }

# %%
# trials = Trials()

# best_hyperparams = fmin(fn = objective,
#                         space = space,
#                         algo = tpe.suggest,
#                         max_evals = 100,
#                         trials = trials)

# %%
# print("The best hyperparameters are : ","\n")
# print(best_hyperparams)


### **Train/Test Proof**


In [ ]:
# Save final, optimized hyper parameters for training final model here
reg = xgb.XGBRegressor(
    booster='gbtree',
    objective='reg:squarederror',
    base_score=0.5,
    n_estimators=10000,  # tuned
    # n_estimators=20,  # tuned
    min_child_weight=3,
    gamma=0,
    learning_rate=0.01,  # tuned
    subsample=0.8,
    colsample_bytree=0.8,
    max_depth=12,
)

reg.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=2000,
)


**Our model is now trained**


<br><br><br>


## **Feature Importance**


In [ ]:
df_fi = pd.DataFrame(
    data=reg.feature_importances_,
    index=reg.feature_names_in_,
    columns=['importance']
)
df_fi.sort_values('importance').tail(10).plot(
    kind='barh', title='Feature Importance')
plt.show()


## **Test forecast on the test set**


In [ ]:
df_test['PREDICTION'] = reg.predict(X_test)
df_test['PREDICTION'] = df_test['PREDICTION'].apply(
    lambda x: 0 if x < 0 else x)  # controlling for negative predictions

df_test = df_test.filter(like='PREDICTION')


In [ ]:
df_test = pd.merge(
    df,
    df_test,
    left_index=True,
    right_index=True,
    how='left',
)
df_test.tail()


In [ ]:
ax = df_test[TARGET].plot(figsize=(20, 10), style='.')
df_test['PREDICTION'].plot(
    ax=ax,
    style='.'
)
plt.legend(['Truth Data', 'Predictions'])
ax.set_title('Raw data and predictions')
plt.show()


## **Calculate root mean squared error**


In [ ]:
df_test = df_test.loc[df_test.index >= test_train_split_date]
score = np.sqrt(mean_squared_error(
    df_test[TARGET], df_test['PREDICTION']))
stdev = df_test[TARGET].std()
normalized_score = score / stdev
print(f'RMSE Score on Test set: {score:0.6f}')
print(f'RMSE / StdDev on Test set: {normalized_score:0.6f}')


## **Calculate Nominal Error**

-   Look at the worst and best predicted weeks


In [ ]:
df_test['abs_error'] = np.abs(df_test[TARGET] - df_test['PREDICTION'])
df_test['error'] = df_test[TARGET] - df_test['PREDICTION']


In [ ]:
df_test_monthly = df_test.resample('M').sum()

# Create a bar chart using seaborn
sns.barplot(
    x=df_test_monthly.index.strftime('%B'),
    y='error',
    data=df_test_monthly,
    width=0.7,
)

# Rotate x-axis labels
plt.xticks(rotation=45)

# Show plot
plt.show()


In [ ]:
# df_test.to_clipboard(excel=True, index=False, header=True


In [ ]:
# stop


<br><br><br><br><br><br>

## **Predicting into the future**

1. Retrain fresh model on all data
2. create forecast / future dataframe to hang our predictions on
3. predict the future
    1. look into the crystal ball


In [ ]:
X_all = df[FEATURES]
y_all = df[TARGET]

reg = xgb.XGBRegressor(
    booster='gbtree',
    # objective = 'reg:linear',
    objective='reg:squarederror',
    base_score=0.5,
    n_estimators=200000,  # tuned
    # n_estimators = 1000,
    learning_rate=0.0001,  # tuned
    # learning_rate = 0.042,
    max_depth=32,  # tuned
    # max_depth = 6,
)
reg.fit(
    X_all,
    y_all,
    eval_set=[(X_all, y_all)],
    verbose=10000,
)


### **Create Forecast (fc\_) dataframe for predictions**

-   replicate dimensionality of the rear window forecast range
    -   defined by user


In [ ]:
# create add forecast (_fc) time range to main df
df_fc = pd.DataFrame(index=fc_date_range,)


def create_df_fc(
        df_left,
        df_right):
    # list of columns to merge
    # cols_to_merge = [df_left.index, ]
    df_left['index'] = df_left.index
    # create a new dataframe by merging the selected columns with NaN-filled columns
    merged_df = pd.concat([df_left['index'], df_right], axis=1)
    merged_df = merged_df.drop(['index'], axis=1)
    # print(merged_df)

    return merged_df


df = create_df_fc(
    df_fc,
    df
)


In [ ]:
fc_start_date = pd.to_datetime(fc_start_date_str)
# split out `df_fc` into its own named object
df_fc = df.query(f'index > "{fc_start_date.strftime("%Y-%m-%d")}"')


In [ ]:
# stop


<br><br><br>


### **Predict the future**


In [ ]:
# df_fc = convert_columns_for_ml(df_fc)
df_fc['PREDICTION'] = reg.predict(df_fc[FEATURES])


In [ ]:
# plot the future
df_fc['PREDICTION'].plot(
    figsize=(10, 5),
    style='--',
    color=color_pal[5],
    ms=1,
    lw=1, title='Future Predictions'
)
plt.show()


## **Saving Model for Later**


In [ ]:
# reg.save_model('model.json')


### **Loading it back up for validation**


In [ ]:
reg_new = xgb.XGBRegressor()
reg_new.load_model('model.json')
# predict the future
df_fc['PREDICTION'] = reg.predict(df_fc[FEATURES])
# plot the future
df_fc['PREDICTION'].plot(
    figsize=(10, 5),
    style='.',
    color=color_pal[5],
    ms=1,
    lw=1, title='Future Predictions'
)


In [ ]:
df_fc.to_clipboard(excel=True, index=False, header=True)


In [ ]:
# stop
